# Import Part

In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import matplotlib.pyplot as plt # used to plot the model graph 
import numpy as np # used for math calculations
import os 
from glob import glob # used to get all the files in a directory
from keras.preprocessing.image import ImageDataGenerator
from PIL import * # used for image manipulation

#for loading and visualizing audio files
import librosa #import librosa
import librosa as lr
import librosa.display # display audio files

from random import shuffle # for shuffling the data

from pathlib import Path
import pathlib
import wave
import contextlib

In [ ]:
from google.colab import drive # Used to access Google Drive
drive.mount('/content/drive') # Mount Google Drive

MessageError: ignored

Downloading the samples from the teacher

In [ ]:
!cd test_sample/wavFile/ && wget https://alg.backprop.fr/data4colab/sounds/en_01.wav 
!cd test_sample/wavFile/ && wget https://alg.backprop.fr/data4colab/sounds/fr_01.wav

# Getting the prepared spectrograms from the drive

In [ ]:
##Getting the spectrograms from the drive
%%shell 
cp ./drive/MyDrive/Spectrograms/train.zip /content/ # copy the train.zip file to the folder
cp ./drive/MyDrive/Spectrograms/test.zip /content/ # copy the test.zip file to the folder

unzip -q '/content/train.zip' -d '/content/' && rm /content/train.zip  # unzip the train.zip file to the folder and remove the zip file
unzip -q '/content/test.zip' -d '/content/' && rm /content/test.zip  # unzip the test.zip file to the folder and remove the zip file

# Global variables

In [ ]:
img_width, img_height = 500, 128 # Setting the image size
batch_size = 32                  # batch size
epochs = 50                      # number of epochs
validation_split = 0.2           # validation split

In [ ]:
all_files = glob('train/*/*.png') # Getting all the spectrograms from the train folder

num_validation = len(all_files) * validation_split # Defining the number of validation files
num_train = len(all_files) - num_validation # Define the number of training images

validation_steps = int(num_validation / batch_size) # Calculating the number of steps for the validation
steps_per_epoch = int(num_train / batch_size) # Calculating the number of steps for the training

print('Steps per Epoch: ' + str(steps_per_epoch)) 
print('Validation steps: ' + str(validation_steps))

# Train and Test definition

In [ ]:
image_data_generator = ImageDataGenerator(validation_split=validation_split,rescale=1./255) # rescale to 0-1

In [ ]:
train_data_generator = image_data_generator.flow_from_directory( # train_data_generator
    './train/',
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode='binary', 
    subset='training',
    color_mode = 'grayscale')

validation_generator = image_data_generator.flow_from_directory( # validation_generator
    './train/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    color_mode='grayscale')

In [ ]:
test_data_generator = image_data_generator.flow_from_directory( # test_data_generator
    './test/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='grayscale')

# Model definition

In [ ]:
keras.backend.clear_session()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.GlobalMaxPool2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',    # val_accuracy
    patience=8,                # patience before stop the train
    verbose=1,                 # verbose
    mode='max',                # mode
    restore_best_weights=True  # restore best weights
)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [ ]:
model.summary()

In [ ]:
model.output

# Fitting the model

In [ ]:
history = model.fit(
    train_data_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping])

In [ ]:
_, accuracy = model.evaluate(test_data_generator)

In [ ]:
print('Accuracy of the model: ' + str(round(accuracy * 100., 1)) + '%')

# Plotting values from the model

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy for '+ str(epochs)+' epochs')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

# Save the model

In [ ]:
!mkdir -p saved_model
model.save('saved_model/model_'+ str(round(accuracy * 100.))+'_accuracy', save_format='h5')

In [ ]:
!mkdir .testingWav
!cp ./drive/MyDrive/EN/TestSample/*.wav ./testingWav/

# Make prediction from the model

In [ ]:
global_files = glob("./*/*/*.png") # get all files frrom the training folder and test folder
shuffle(global_files) # shuffle the dataset to randomize 
random_file = global_files[0] # choose a random file from all the dataset
print(random_file)


img = tf.keras.utils.load_img((random_file), target_size=(500, 128),color_mode='grayscale') # load the image
img_array = tf.keras.utils.img_to_array(img)/255 # convert the image to an array
img_array = tf.expand_dims(img_array,0) # add a dimension to the array

predict = model.predict(img_array) # Make a prediction on the random image
label = predict[0][0] # Get the value of the prediction

# More is close to 0, more the prediction is an English conversation
# Less is close to 1, more the prediction is a French conversation
print('Value: ' + str(label)) 

if(label <= 0.25): 
  print("You're speaking mostly in English")
elif(label > 0.75): 
  print("You're speaking mostly in French") 
elif(label > 0.25 and label <= 0.50): 
  print("It's look like you're speaking in English")
elif(label > 0.50 and label <= 0.75): 
  print("It's look like you're speaking in French")

# Other functions

In [ ]:
!mkdir saved_model
!cp ./drive/MyDrive/Models/*.h5 ./saved_model/

In [ ]:
!mkdir -p ./test_sample/wavFile/ ./test_sample/pngFile/
!cp ./drive/MyDrive/EN/TestSample/*.wav ./test_sample/wavFile/

In [ ]:
def getResult(label):
  if(label <= 0.25): 
    percentage = 100 - (label * 100)
    result = 'You\'re speaking mostly in English at ' + str(round(percentage, 3)) +'%'
  elif(label > 0.75): 
    percentage = label * 100
    result = 'You\'re speaking mostly in French at ' + str(round(percentage, 3)) +'%'
  elif(label > 0.25 and label <= 0.50):
    percentage = 100 - (label * 100)
    result = 'It\'s look like you\'re speaking in English at '  + str(round(percentage, 3)) +'%'
  elif(label > 0.50 and label <= 0.75):
    percentage = label * 100
    result = 'It\'s look like you\'re speaking in French at ' + str(round(percentage, 3)) +'%'
  return result

In [ ]:
def createSpectrogram(audio_segment):
    
    h1 = audio_segment.shape[0] // img_width
    spec = lr.feature.melspectrogram(audio_segment, n_mels=img_height, hop_length=int(h1))

    image = lr.core.power_to_db(spec)

    image_np = np.asmatrix(image)

    image_np_scaled_temp = (image_np - np.min(image_np))

    image_np_scaled = image_np_scaled_temp / np.max(image_np_scaled_temp)

    return image_np_scaled[:, 0:img_width]

In [ ]:
def makeSpectrogramOfFolder(path_folder):
  # Define Global variables

  directorySample = path_folder
  outputSample = "./test_sample/pngFile/"

  directories = [directorySample]
  outputs = [outputSample]

  for directory in directories:
    for filename in os.listdir(directory):
      filetitle, _ = os.path.splitext(filename)
      audio_segment, sample_rate = librosa.load(directory+filename)
      a = createSpectrogram(audio_segment)
      for i in range(0,len(outputs)):
        if directory == directories[i]:
          plt.imsave(outputs[i]+filetitle+".png", a, cmap='gray')

In [ ]:
def makeSpectrogramOfFile(file_path):
  # Define Global variables
  filename = './test_sample/wavFile/en_sample_n3.wav'
  output = "./test_sample/pngFile/"

  filetitle = Path(file_path).stem
  audio_segment, sample_rate = librosa.load(file_path)
  a = createSpectrogram(audio_segment)
  path = plt.imsave(output+filetitle+".png", a, cmap='gray')

  png_path = output+filetitle+".png"
  return png_path

In [ ]:
def predictRandomImg(path_model):
  loaded_model = tf.keras.models.load_model(path_model)
  
  global_files = glob("./*/*/*.png") # get all files from the training folder and test folder
  shuffle(global_files) # shuffle the dataset to randomize 
  random_file = global_files[0] # choose a random file from all the dataset

  img = tf.keras.utils.load_img((random_file), target_size=(500, 128),color_mode='grayscale') # load the image
  img_array = tf.keras.utils.img_to_array(img)/255 # convert the image to an array
  img_array = tf.expand_dims(img_array,0) # add a dimension to the array

  predict = loaded_model.predict(img_array) # Make a prediction on the random image
  label = predict[0][0] # Get the value of the prediction
  
  return getResult(label) 

In [ ]:
def predictSpecificImg(image, path_model):
  loaded_model = tf.keras.models.load_model(path_model)

  img = tf.keras.utils.load_img((image), target_size=(500, 128),color_mode='grayscale') # load the image
  img_array = tf.keras.utils.img_to_array(img)/255 # convert the image to an array
  img_array = tf.expand_dims(img_array,0) # add a dimension to the array

  predict = loaded_model.predict(img_array) # Make a prediction on the random image
  label = predict[0][0] # Get the value of the prediction
  return getResult(label)

In [ ]:
def predictFromWav(wav_path, path_model):
 return predictSpecificImg(makeSpectrogramOfFile(wav_path), path_model)

In [ ]:
# split a wav file of X seconds into smaller wav files of 8 seconds
def split_wavFile(wav_path, seconds):
    wav_file = wave.open(wav_path, 'r')     # open the wav file
    frames = wav_file.getnframes()          # get the number of frames
    rate = wav_file.getframerate()          # get the frame rate
    duration = frames / float(rate)         # get the duration

    if duration < seconds:
        error = 'Cannot split the wav file because it is less than 8 seconds, please choose an another wavFile'
        return error

    elif duration > seconds:
      num_chunks = int(duration / seconds)    # get the number of chunks
      filetitle = Path(wav_path).stem        # get the file title    

      parent_folder = str(pathlib.Path(wav_path).parent.resolve()) + '/'
      folder_name = str(Path(wav_path).stem) + '_Splitted/' 
      os.mkdir(parent_folder + folder_name)

      for i in range(num_chunks):             # iterate over the number of chunks
          start = i * seconds * rate          # get the start frame
          end = (i + 1) * seconds * rate      # get the end frame
          wav_file.setpos(start)              # set the position in the file
          frames = wav_file.readframes(int(end - start))          # read the frames
          out_file = wave.open(parent_folder + str(folder_name) + str(filetitle) + '_' + str(i) + '.wav', 'w')    # open the output file
          out_file.setparams(wav_file.getparams())                # set the output file parameters
          out_file.writeframes(frames)        # write the frames to the output file
          out_file.close()                    # close the output file
      wav_file.close()                        # close the input file

In [ ]:
def getWavLength(wav_path):
  with contextlib.closing(wave.open(wav_path,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
    result = round(duration,2)
    s = 'File: ' + str(wav_path) + ' |  Length: ' + str(result)
    return s + 's'

# Testing the functions

Make all the spectrograms of the wavfolder

In [ ]:
#makeSpectrogramOfFolder('./test_sample/wavFile/en_01_Splitted/')

Predict a random image from the global dataset

In [ ]:
#predictRandomImg('./saved_model/model_99_accuracy.h5')

Predict a specific image 

In [ ]:
#predictSpecificImg('./test_sample/pngFile/yes.png', './saved_model/model_99_accuracy.h5')

Predict from a wavFile

In [ ]:
#predictFromWav('./test_sample/wavFile/en_01_Splitted/en_01_1.wav', './saved_model/model_99_accuracy.h5')

Splitting the Wav file if it exceed 8 sec

In [ ]:
#split_wavFile('./test_sample/wavFile/en_01.wav', 8)

Getting the length of the wavFile

In [ ]:
#getWavLength('./test_sample/wavFile/en_01.wav')